In [ ]:
### Network analysis
# Centralities
# Clusters
# Bot detection

In [7]:
import pandas as pd

In [13]:
data = pd.read_pickle("preprocessed_data.pkl")

Bot detection

In [14]:
data["NodeType"].value_counts()

Neutral        96528
Unknown        37371
pro_Ukraine    27098
pro_Russian      559
Both              72
Name: NodeType, dtype: int64

In [53]:
pro_UkraineBefore = data["userid"][(data["NodeType"] == "pro_Ukraine") & (data["tweetcreatedts"] <= "2022-03-04 23:59:59+00:00")].sample(100)
pro_UkraineAfter = data["userid"][(data["NodeType"] == "pro_Ukraine") & (data["tweetcreatedts"] > "2022-03-04 23:59:59+00:00")].sample(100)
pro_RussianBefore = data["userid"][(data["NodeType"] == "pro_Russian") & (data["tweetcreatedts"] <= "2022-03-04 23:59:59+00:00")].sample(100)
pro_RussianAfter = data["userid"][(data["NodeType"] == "pro_Russian") & (data["tweetcreatedts"] > "2022-03-04 23:59:59+00:00")].sample(100)

groups = {"pro_UkraineBefore": pro_UkraineBefore, "pro_UkraineAfter": pro_UkraineAfter, "pro_RussianBefore":pro_RussianBefore, "pro_RussianAfter":pro_RussianAfter}

In [55]:
### Get results from botometer
# More info: https://github.com/IUNetSci/botometer-python

# !pip3 install botometer
# !pip3 install requests tweepy #dependencies for botometer
import botometer
from config import *

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)
group_results = dict()
for group in groups.items():
    accounts = group[1]
    results = []     
    for screen_name, result in bom.check_accounts_in(accounts):
        results.append(result)    
    group_results[group[0]] = results

In [58]:
# Check results. If bot: add results to the attributes
bot_scores = dict()
bot_score_means = dict()
for group, results in group_results.items():    
    non_existing = 0
    sum_en = 0
    sum_un = 0
    for result in results:
        try: 
            english_benchmark = result["cap"]["english"]
            universal_benchmark = result["cap"]["universal"]
            english_score = result["raw_scores"]["english"]["overall"]
            universal_score = result["raw_scores"]["universal"]["overall"]

            bot_en = False
            if english_score >= english_benchmark:
                bot_en = True
                
            bot_uni = False
            if universal_score >= universal_benchmark:
                bot_uni = True
                
            if bot_en or bot_uni:
                english_all_scores = result["raw_scores"]["english"]
                universal_all_scores = result["raw_scores"]["universal"]
                print("I'm a bot")
                # add bot scores attribute to the node
                bot_scores[result["user"]["user_data"]["id_str"]] = {"en": english_all_scores, "un": universal_all_scores}
            
            sum_en += english_score
            sum_un += universal_score

        except:
            non_existing += 1 
            continue
    
    mean_en = sum_en/(len(results)-non_existing)
    mean_un = sum_un/(len(results)-non_existing)
    
    bot_score_means[group] = {"en": mean_en, "un": mean_un}
    
    print("Group:", group)
    print("Mean score EN:",round(mean_en, 4),"Mean score UNIVERSAL:",round(mean_un, 4))
    
# example result: {'cap':
#   {'english': 0.8995513244218455, 'universal': 0.8733944954488508},
#       'display_scores':
#           {'english': {'astroturf': 0.8, 'fake_follower': 1.9, 'financial': 0.2, 'other': 4.7, 'overall': 4.7, 'self_declared': 4.6, 'spammer': 0.8},
#           universal': {'astroturf': 0.8, 'fake_follower': 1.2, 'financial': 0.2, 'other': 4.4, 'overall': 4.6, 'self_declared': 4.6, 'spammer': 0.8}},
#       'raw_scores':
#           {'english': {'astroturf': 0.17, 'fake_follower': 0.38, 'financial': 0.03, 'other': 0.94, 'overall': 0.94, 'self_declared': 0.93, 'spammer': 0.16},
#           'universal': {'astroturf': 0.15, 'fake_follower': 0.24, 'financial': 0.04, 'other': 0.87, 'overall': 0.91, 'self_declared': 0.91, 'spammer': 0.15}},
#   'user': {'majority_lang': 'en', 'user_data': {'id_str': '3380828067', 'screen_name': 'dsn_status'}}}

Group: pro_UkraineBefore
Mean score EN: 0.07 Mean score UNIVERSAL: 0.04
Group: pro_UkraineAfter
Mean score EN: 0.13 Mean score UNIVERSAL: 0.16
Group: pro_RussianBefore
Mean score EN: 0.12 Mean score UNIVERSAL: 0.13


ZeroDivisionError: division by zero